In [1]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import IPython.display as ipd
from scipy.io.wavfile import write

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

def load_data_from_json(file_path):
    with open(file_path, 'r') as file:
        test_text_sem_dic = json.load(file)
    for key in test_text_sem_dic:
        test_text_sem_dic[key] = torch.tensor(test_text_sem_dic[key])
    return test_text_sem_dic

def ensure_directory_exists(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1051)
           2	LOAD_FAST(arg=0, lineno=1054)
           4	LOAD_CONST(arg=1, lineno=1054)
           6	BINARY_SUBSCR(arg=None, lineno=1054)
           8	LOAD_FAST(arg=0, lineno=1054)
          10	LOAD_CONST(arg=2, lineno=1054)
          12	BINARY_SUBSCR(arg=None, lineno=1054)
          14	COMPARE_OP(arg=4, lineno=1054)
          16	LOAD_FAST(arg=0, lineno=1054)
          18	LOAD_CONST(arg=1, lineno=1054)
          20	BINARY_SUBSCR(arg=None, lineno=1054)
          22	LOAD_FAST(arg=0, lineno=1054)
          24	LOAD_CONST(arg=3, lineno=1054)
          26	BINARY_SUBSCR(arg=None, lineno=1054)
          28	COMPARE_OP(arg=5, lineno=1054)
          30	BINARY_AND(arg=None, lineno=1054)
          32	RETURN_VALUE(arg=None, lineno=1054)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:state.pc_initial: State(pc_initial=0 nstack_

## LJ Speech

In [2]:

# choose pretrained model and the trained step
# model = 'ljs_base'
# model = 'onehour_ljs_base'
# model = 'librif_base'
model = 'emovdb_base_pretrained16'
# step = 'G_50000'
# step = "G_100000"
# step = "G_150000"
# step = "G_200000"
# step = "G_250000"
step = "G_300000"


common_dir = '/data/vitsGPT/vits/'
log_dir = f'{common_dir}ori_vits/logs/'
save_dir = f'{log_dir}{model}/{step}/source_model_test_wav'
ensure_directory_exists(save_dir)
hps = utils.get_hparams_from_file(f"{log_dir}{model}/config.json")

# Dictionary to map the model to its corresponding test_text_sem_dic_file
model_to_test_text_sem_dic = {
    'ljs_base': 'ljs_text_sem_ave_5120.json',
    'onehour_ljs_base': 'ljs_text_sem_ave_5120.json',
    'librif_base': 'librif_text_sem_ave_5120.json',
    'emovdb_base_pretrained16': 'emovdb_text_sem_ave_5120.json',
}
# Get the corresponding test_text_sem_dic_file for the chosen model
test_text_sem_dic_file = model_to_test_text_sem_dic[model]
test_text_sem_dic = load_data_from_json(f"{common_dir}filelists/{test_text_sem_dic_file}")

In [3]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint(f"{log_dir}{model}/{step}.pth", net_g, None)

In [4]:
keys = list(test_text_sem_dic.keys())
range_limit = len(keys)
range_limit = min(range_limit, len(keys))
print(f"range_limit: {range_limit}")
m = 5

for i in range(range_limit):
    key = keys[i]  # key 是字符串
    s = get_text(key, hps)  # 将 key 赋值给 s
    print(key)
    with torch.no_grad():
        x_tst = s.cuda().unsqueeze(0)
        x_tst_lengths = torch.LongTensor([s.size(0)]).cuda()     
        print(f"x_tst: {x_tst.shape}")
        print(f"x_tst_lengths: {x_tst_lengths}")  
        audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
    if i < m:
        ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
    write(f"{save_dir}/output_ori_{i}.wav", hps.data.sampling_rate, audio)

range_limit: 51
Pearce's little eyes were fixed on him shrewdly.
x_tst: torch.Size([1, 101])
x_tst_lengths: tensor([101], device='cuda:0')


I came for information more out of curiosity than anything else.
x_tst: torch.Size([1, 147])
x_tst_lengths: tensor([147], device='cuda:0')


They closed now until his fingers were like cords of steel.
x_tst: torch.Size([1, 129])
x_tst_lengths: tensor([129], device='cuda:0')


He saw Jeanne falter for a moment.
x_tst: torch.Size([1, 77])
x_tst_lengths: tensor([77], device='cuda:0')


Surely I will excuse you, she cried.
x_tst: torch.Size([1, 83])
x_tst_lengths: tensor([83], device='cuda:0')


In a flash Philip followed its direction.
x_tst: torch.Size([1, 83])
x_tst_lengths: tensor([83], device='cuda:0')
He would first hunt up Gregson and begin his work there.
x_tst: torch.Size([1, 121])
x_tst_lengths: tensor([121], device='cuda:0')
What was the object of your little sensation.
x_tst: torch.Size([1, 93])
x_tst_lengths: tensor([93], device='cuda:0')
The gray eyes faltered; the flush deepened.
x_tst: torch.Size([1, 85])
x_tst_lengths: tensor([85], device='cuda:0')
Then, and at supper, he tried to fathom her.
x_tst: torch.Size([1, 91])
x_tst_lengths: tensor([91], device='cuda:0')
A shadow was creeping over Pierre's eyes.
x_tst: torch.Size([1, 81])
x_tst_lengths: tensor([81], device='cuda:0')
The singing voice approached rapidly.
x_tst: torch.Size([1, 71])
x_tst_lengths: tensor([71], device='cuda:0')
He had barely entered this when he saw the glow of a fire.
x_tst: torch.Size([1, 117])
x_tst_lengths: tensor([117], device='cuda:0')
Perhaps she had already met her fate a little d